Many steps copied from Tanisha Harde on Kaggle

#### Cite later!!!

Tanisha Harde on Kaggle
https://www.kaggle.com/code/tanishaharde/e-commerce-shopper-purchase-prediction-analysis/notebook

xxKavosxx
https://www.kaggle.com/code/xxkavosxx/xgboost-0-92-roc-pipeline

## Preliminaries

In [348]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

%matplotlib inline     

In [349]:
df=pd.read_csv('online_shoppers_intention.csv')

In [350]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [ ]:
# Turn booleans into numeric

df['Weekend'] = df['Weekend'].astype(int)
df['Revenue'] = df['Revenue'].astype(int)

In [ ]:
# show unique values of Month

len(df['Month'].unique())

10

## Modified Tanisha

#### Processing

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder

# Manual label encoding for month
df['Month'] = df['Month'].map(
    {'Feb': 2, 
     'Mar': 3, 
     'May': 5,
     'June':6,
     'Jul': 7,
     'Aug': 8,
     'Sep': 9,
     'Oct':10,
     'Nov':11,
     'Dec':12})

     
mmax = MinMaxScaler()
df['Month'] = mmax.fit_transform(df['Month'].values.reshape(-1,1))

print(df['Month'].unique())


# Encoding for categoricals
cols_to_encode = [
    # 'OperatingSystems',
    # 'Browser',
    # 'Region',
    # 'TrafficType',
    'VisitorType'
]
le = LabelEncoder()
df[cols_to_encode] = df[cols_to_encode].apply(le.fit_transform)

# use pd.get_dummies for onehot encoding



# Deduplicate 
df.drop_duplicates(inplace=True)

# Create new numerical variables

df['TotalPages'] = df['Administrative'] + df['Informational'] + df['ProductRelated']
df['TotalTime'] = df['Administrative_Duration'] + df['Informational_Duration'] + df['ProductRelated_Duration']
df['Adm_time_per_page'] = (df['Administrative_Duration']/df['Administrative']).fillna(0)
df['Info_time_per_page'] = (df['Informational_Duration']/df['Informational']).fillna(0)
df['Pr_time_per_page'] = (df['ProductRelated_Duration']/df['ProductRelated']).fillna(0)
# df['Adm_proportion'] = (df['Administrative']/df['TotalPages']).fillna(0)
# df['Info_proportion'] = (df['Informational']/df['TotalPages']).fillna(0)
# df['Pr_proportion'] = (df['Administrative']/df['TotalPages']).fillna(0)


# Minmaxscale on numericals
cols_to_scale = [
    # 'Adm_proportion',
    # 'Info_proportion', 
    # 'Pr_proportion',
    'Administrative_Duration',
    'Informational_Duration',
    'ProductRelated_Duration',
    'TotalPages',
    'TotalTime',
    'Adm_time_per_page',
    'Info_time_per_page',
    'Pr_time_per_page',
    'BounceRates',
    'ExitRates',
    'PageValues',
    'SpecialDay']
scaler = PowerTransformer()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])



# Drop categories of pages visited
df = df.drop([
    'Administrative',
    'Informational', 
    'ProductRelated',
    # 'Administrative_Duration',
    # 'Informational_Duration',
    # 'ProductRelated_Duration'
    ], axis=1)


[0.  0.1 0.3 0.8 0.4 0.5 0.6 0.9 0.7 1. ]


In [ ]:
X=df.drop('Revenue',axis=1)
y=df['Revenue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:

# # Apply SMOTE on the encoded features and target variable
# smote = SMOTE(sampling_strategy='minority')
# X_train, y_train = smote.fit_resample(X_train, y_train)

# # Convert the resampled target variable to a pandas Series
# y_train = pd.Series(y_train)


#### Modelling

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf = RandomForestClassifier(n_estimators=100, random_state=1)
rf.fit(X_train, y_train)

# Make predictions on the test set and evaluate model performance
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

Accuracy: 0.8951249487914789
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      2059
           1       0.73      0.52      0.61       382

    accuracy                           0.90      2441
   macro avg       0.82      0.74      0.77      2441
weighted avg       0.89      0.90      0.89      2441



In [ ]:
# confusion matrix

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm

array([[1987,   72],
       [ 184,  198]])

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_estimators=100, random_state=42)
et.fit(X_train, y_train)
y_pred = et.predict(X_test)
accuracy_et = accuracy_score(y_test, y_pred)
print('Extra Trees Accuracy:', accuracy_et)

print(classification_report(y_test, y_pred))

Extra Trees Accuracy: 0.9000409668168783
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2059
           1       0.74      0.56      0.64       382

    accuracy                           0.90      2441
   macro avg       0.83      0.76      0.79      2441
weighted avg       0.89      0.90      0.89      2441



In [347]:
from xgboost import XGBClassifier


xgb = XGBClassifier(n_estimators=100, random_state=42)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred)
print('XGBoost Accuracy:', accuracy_xgb)

print(classification_report(y_test, y_pred))

XGBoost Accuracy: 0.8877509217533798
              precision    recall  f1-score   support

           0       0.92      0.95      0.93      2059
           1       0.68      0.53      0.60       382

    accuracy                           0.89      2441
   macro avg       0.80      0.74      0.77      2441
weighted avg       0.88      0.89      0.88      2441



In [ ]:
from sklearn.tree import export_graphviz
import graphviz
from IPython.display import display

str_tree = export_graphviz(rf.estimators_[0], 
   out_file=None, 
   feature_names=X_train.columns, # column names
   filled=True,        
   special_characters=True, 
   rotate=True)

graph = graphviz.Source(str_tree)

graph.render()


'Source.gv.pdf'

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=42)

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred)
print('Logistic Regression Accuracy:', accuracy_lr)

print(classification_report(y_test, y_pred))

Logistic Regression Accuracy: 0.8865219172470299
              precision    recall  f1-score   support

           0       0.92      0.95      0.93      2059
           1       0.66      0.57      0.61       382

    accuracy                           0.89      2441
   macro avg       0.79      0.76      0.77      2441
weighted avg       0.88      0.89      0.88      2441



In [ ]:
# with lasso regularization

lr = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred)
print('Logistic Regression Accuracy:', accuracy_lr)

print(classification_report(y_test, y_pred))

Logistic Regression Accuracy: 0.8861122490782466
              precision    recall  f1-score   support

           0       0.92      0.95      0.93      2059
           1       0.66      0.57      0.61       382

    accuracy                           0.89      2441
   macro avg       0.79      0.76      0.77      2441
weighted avg       0.88      0.89      0.88      2441

